In [1]:
import os
from datetime import datetime

from typing import Type, Optional, Union, Tuple

import numpy as np
import pandas as pd
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from skopt import gp_minimize

import matplotlib.pyplot as plt 
plt.rcParams['figure.facecolor'] = "white"

from reservoir import reservoir as res
from data import data
from error import error

In [2]:
# define constants 
LAMBDA = 0.9056     # lorenz lyapunov exponent
SEED = 42
STATE = np.random.RandomState(SEED)
RES_DIM = 100
ALPHA = 0.001

In [3]:
# reservoir parameters
hyperparams = {
    'GAMMA': 7.7,
    'SIGMA': 0.81,
    'RHO_IN': 0.37,
    'K': 3,
    'RHO_R': 0.41,
    'N': 100
}

In [4]:
simulation_parameters = {
    "DEL_T": 0.02,          # time step size
    "STEPS": 30000,         # total steps
    "WASHOUT": 10000,       # washout 
    # n = STEPS - WASHOUT
    "ALPHA": 0.001,         # Tikhonov regularisation constant
    "d": 3,
    "INITIAL_STATE_RANGE": 5
}

With all else held constant, vary the number of nodes and see effect. 

In [5]:
def single_trial(training_data, test_data, hyperparams, simulation_parameters, 
               state=STATE, adjust_for_symmetry=True):
    """
    Args:
        hyperparams (dict): dictionary of hyperparameters.
        shapes (tuple): dimensions of simulation, in the following order:
            N (int): dimension of a reservoir state
            d (int): dimension of input signal

    """
    # dims
    n_train = training_data.shape[0]
    n_test = test_data.shape[0]
    N = hyperparams["N"]
    d = training_data.shape[1]

    # simulation parameters
    del_t = simulation_parameters["DEL_T"]
    alpha = simulation_parameters["ALPHA"]
    
    # generate reservoir inputs
    print("\t\tGenerating reservoir inputs...", end="")
    W_in = res.generate_W_in(hyperparams, (N, d), state)
    W_r = res.generate_W_r(hyperparams, (N, N), state)
    print("complete.")

    # generate training reservoir
    print("\t\tGenerating training reservoir...", end="")
    training_res = res.generate_training_reservoir(
        data=training_data,
        hyperparams=hyperparams,
        W_r=W_r,
        W_in=W_in,
        delta_t=del_t,
        adjust_for_symmetry=adjust_for_symmetry
    )
    print("complete.")

    # generate output transformation
    print("\t\tGenerating output transformation...", end="")
    W_out = res.generate_W_out(
        data=training_data,
        res=training_res,
        alpha=alpha
    )
    print("complete.")

    # generate forecast reservoir
    print("\t\tGenerating forecast reservoir...", end="")
    forecast_res = res.generate_forecast_reservoir(
        r_0=np.dot(W_in, test_data[0]),
        data=test_data,
        hyperparams=hyperparams,
        W_r=W_r,
        W_in=W_in,
        W_out=W_out,
        delta_t=del_t,
        adjust_for_symmetry=adjust_for_symmetry
    )
    print("complete.")

    # make prediction
    print("\t\tPredicting...", end="")
    preds = res.readout_network(res=forecast_res, W_out=W_out)
    print("complete.")

    # goodness of forecast
    print("\t\tComputing loss...", end="")
    losses = [
        # L2 error
        np.linalg.norm(error.RMSE(test_data, preds)),
        # Error over one Lyapunov time (Griffith et al)
        error.griffith_epsilon_1(test_data, preds, del_t, LAMBDA)
    ]
    print("complete.")

    return {
        "losses": losses,
        "preds": preds,
        "forecast_res": forecast_res,
        "training_res": training_res,
        "W_out": W_out,
        "W_in": W_in,
        "W_r": W_r
    }
    

In [6]:
simulation_parameters = {
    "DEL_T": 0.02,          # time step size
    "STEPS": 30000,         # total steps
    "WASHOUT": 10000,       # washout 
    # n = STEPS - WASHOUT
    "ALPHA": 0.001,         # Tikhonov regularisation constant
    "d": 3
}

def generate_initial_states(num_trials, simulation_parameters, 
                            state: np.random.RandomState = STATE):
    d = simulation_parameters["d"]
    initial_state_range = simulation_parameters["INITIAL_STATE_RANGE"]
    
    training_initial_states = state.uniform(-initial_state_range, initial_state_range, (num_trials, d))
    test_initial_states = state.uniform(-initial_state_range, initial_state_range, (num_trials, d))

    return training_initial_states, test_initial_states
            

def many_trials(num_trials, hyperparams, simulation_parameters, 
                training_data_ls, test_data_ls, 
                state: np.random.RandomState=STATE, adjust_for_symmetry=True):
    del_t = simulation_parameters["DEL_T"]
    steps = simulation_parameters["STEPS"]
    washout = simulation_parameters["WASHOUT"]

    results = []
    for i in range(num_trials):
        training_data = training_data_ls[i]
        test_data = test_data_ls[i]
        
        # generate data
        results.append(single_trial(
            training_data=training_data,
            test_data=test_data,
            hyperparams=hyperparams,
            simulation_parameters=simulation_parameters,
            state=state,
            adjust_for_symmetry=adjust_for_symmetry
        ))
        print("\tTrials performed: {} / {}".format(i+1, num_trials))

    return results

# TESTING

In [7]:
NUM_TRIALS = 3

# reservoir parameters
hyperparams = {
    'GAMMA': 7.7,
    'SIGMA': 0.81,
    'RHO_IN': 0.37,
    'K': 3,
    'RHO_R': 0.41,
    'N': 100
}

simulation_parameters = {
    "DEL_T": 0.02,          # time step size
    "STEPS": 30000,         # total steps
    "WASHOUT": 10000,       # washout 
    # n = STEPS - WASHOUT
    "ALPHA": 0.001,         # Tikhonov regularisation constant
    "d": 3,
    "INITIAL_STATE_RANGE": 5
}

# N_range = [100 * (2 ** n) for n in range(6)]
N_range = [100, 200]

sims = []

print("Beginning simulations...")

# generate initial states
print("\tGenerating initial states...", end="")
training_initial_states, test_initial_states = generate_initial_states(
    num_trials=NUM_TRIALS,
    simulation_parameters=simulation_parameters,
    state=STATE
)
print("complete.")

# generate data
training_data_ls = []
test_data_ls = []

print("\tGenerating training and test data...", end="")
for i in range(NUM_TRIALS):
    training_data_ls.append(data.generate_lorenz_63(
        initial_state=training_initial_states[i],
        del_t=simulation_parameters["DEL_T"],
        steps=simulation_parameters["STEPS"],
        washout=simulation_parameters["WASHOUT"]
    ))

    test_data_ls.append(data.generate_lorenz_63(
        initial_state=test_initial_states[i],
        del_t=simulation_parameters["DEL_T"],
        steps=simulation_parameters["STEPS"],
        washout=simulation_parameters["WASHOUT"]
    ))
print("complete.")

count = 0
for res_dim in N_range:
    hyperparams["N"] = res_dim
    sims.append(many_trials(
        num_trials=NUM_TRIALS,
        hyperparams=hyperparams,
        training_data_ls=training_data_ls,
        test_data_ls=test_data_ls,
        simulation_parameters=simulation_parameters,
        state=STATE,
        adjust_for_symmetry=True
    ))

    print("Simulations done: {} / {}".format(count + 1, len(N_range)))
    count += 1
print("Simulations complete.")


Beginning simulations...
	Generating initial states...complete.
	Generating training and test data...complete.
		Generating reservoir inputs...complete.
		Generating training reservoir...complete.
		Generating output transformation...complete.
		Generating forecast reservoir...complete.
		Predicting...complete.
		Computing loss...complete.
	Trials performed: 1 / 3
		Generating reservoir inputs...complete.
		Generating training reservoir...complete.
		Generating output transformation...complete.
		Generating forecast reservoir...complete.
		Predicting...complete.
		Computing loss...complete.
	Trials performed: 2 / 3
		Generating reservoir inputs...complete.
		Generating training reservoir...complete.
		Generating output transformation...complete.
		Generating forecast reservoir...complete.
		Predicting...complete.
		Computing loss...complete.
	Trials performed: 3 / 3
Simulations done: 1 / 2
		Generating reservoir inputs...complete.
		Generating training reservoir...complete.
		Generatin

A function to store our results.

In [9]:
today = datetime.today().strftime("%Y%m%d")

In [22]:
def make_dir(dir):
    try:
        os.mkdir(dir)
        print("Created directory: {}".format(dir))
        return True
    except FileExistsError:
        return False

In [23]:
dir = "results/node_lengths/{}.{}.{}/".format(today, N_range[0], N_range[-1])
make_dir(dir)

i = 0
for sim in sims:
    sim_dir = dir + str(N_range[i]) + "/"
    make_dir(sim_dir)
    
    num_trials = len(sim)
    j = 1
    for trial in sim:
        trial_dir = sim_dir + str(j) + "/"
        make_dir(trial_dir)

        print("Writing results...")
        with open(trial_dir + "losses.out", 'w') as f:
            f.write(str(trial["losses"])[1:-1])
        print("\tWrote {}.out".format(trial_dir + key))
        
        for key in trial.keys():
            if key != "losses":
                np.savetxt("{}.out".format(trial_dir + key), trial[key], delimiter=",")
                print("\tWrote {}.out".format(trial_dir + key))

        print("Writing complete.")
        j += 1

    i += 1

Created directory: results/node_lengths/20220717.100.200/
Created directory: results/node_lengths/20220717.100.200/100/
Created directory: results/node_lengths/20220717.100.200/100/1/
Writing results...
	Wrote results/node_lengths/20220717.100.200/100/1/training_res.out
	Wrote results/node_lengths/20220717.100.200/100/1/preds.out
	Wrote results/node_lengths/20220717.100.200/100/1/forecast_res.out
	Wrote results/node_lengths/20220717.100.200/100/1/training_res.out
	Wrote results/node_lengths/20220717.100.200/100/1/W_out.out
	Wrote results/node_lengths/20220717.100.200/100/1/W_in.out
	Wrote results/node_lengths/20220717.100.200/100/1/W_r.out
Writing complete.
Created directory: results/node_lengths/20220717.100.200/100/2/
Writing results...
	Wrote results/node_lengths/20220717.100.200/100/2/W_r.out
	Wrote results/node_lengths/20220717.100.200/100/2/preds.out
	Wrote results/node_lengths/20220717.100.200/100/2/forecast_res.out
	Wrote results/node_lengths/20220717.100.200/100/2/training_res

In [ ]:
# for sim in sims:
#     for trial in sim:
#         print(trial.keys())
#         for key in trial.keys():
#             print(str(type(trial[key])))
#         print()

dict_keys(['losses', 'preds', 'forecast_res', 'training_res', 'W_out', 'W_in', 'W_r'])
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

dict_keys(['losses', 'preds', 'forecast_res', 'training_res', 'W_out', 'W_in', 'W_r'])
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

dict_keys(['losses', 'preds', 'forecast_res', 'training_res', 'W_out', 'W_in', 'W_r'])
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

dict_keys(['losses', 'preds', 'forecast_res', 'training_res', 'W_out', 'W_in', 'W_r'])
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

dict_keys(['